# GPT batch runner; OLCI L1 to L2 (C2RCC & IdePix)

    Version: 2.0
    Date:    10/04/2019
    Author:  Ben Loveday and Hayley Evers-King (Plymouth Marine Laboratory)
    Credit:  This code was developed for EUMETSAT under contracts for the Copernicus 
             programme.
    License: This code is offered as free-to-use in the public domain, with no warranty.

This routine shows an example of how to use python to call SNAP's GPT processor, to allow automated processing of S3A OLCI images from L1 to L2. GPT runs according to a 'graph' that can be built in SNAP and is stored as a .xml file. The example graph here performs three tasks:

    1. Subsets the input data (to save time)
    2. Runs to IdePix cloud processor on the subset data
    3. Runs the C2RCC processor on the subset data
    4. Merges the IdePix and C2RCC output into a single netcdF output file

However, this basic script can be used/adapted to run any graph using SNAPs capability from the command line. With minimal adaptation this script can be re-written for calling through an automated shell script (in Linux), rather than being called through Jupyter. Here, we will only consider the Jupyter context, as it works across all platforms.

We begin by importing some python modules

In [1]:
import os, shutil, subprocess
from os import path, listdir, system
import fnmatch
import fileinput
import platform

Next we have to tell our code where the GPT executable is. The default path is hown below, but you may have to adapt this depending on where you installed SNAP.

In [2]:
# set up the environment, path to your data
# e.g. MYPATH = os.path.join("C:/","Users","me","Desktop")
MYPATH = os.path.join("C:/","Users","docle","Desktop","OSOS_notebooks") # this needs to be changed to your directory

# set up the environment, path to your gpt processor
# e.g. GPTPATH = os.path.join("C:/","Program Files","snap","bin","gpt.exe")
GPTPATH = os.path.join("C:/","Program Files","snap","bin","gpt.exe")

Quick adaptation of the path to make sure it works on windows

In [3]:
# accommodate spaces in windows paths....
if platform.system().lower() == 'windows':
    GPTPATH = '"' + GPTPATH + '"'

Now we must point the code to our config file (in the same directory as this file), and the data we downloaded. It is the configuration file that describes the processes that SNAP will perform in our automated chain.

In [4]:
# define your graph xml, and input and output directory paths
template_xml = 'config_template.xml'
input_dir = os.path.join(MYPATH,'GPT_L1_test_data')
output_dir = os.path.join(os.getcwd(),'GPT_L2_C2RCC_OUTPUT')

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

Now we search the input directory to find Level-1 Sentinel-3 data

In [5]:
# make a list of all the input files in your input directory
input_files=[]
for root, _, filenames in os.walk(input_dir):
    for filename in fnmatch.filter(filenames, '*xfdumanifest.xml'):
        input_files.append(os.path.join(root, filename))
        
# and show the list        
for input_file in input_files:
    print(input_file)

C:/Users\docle\Desktop\ocean-olly_wekeo_changes\GPT_L1_test_data\S3A_OL_1_EFR____20181027T082308_20181027T082608_20181028T140235_0179_037_192_3420_MAR_O_NT_002.SEN3\xfdumanifest.xml


And finally, we start the main part of our code, where we call GPT to process this L1 data. 

In [7]:
# MAIN
# this is the loop where the magic happens :-)
# the loop goes over each input file in the input_files list
for input_file in input_files:
    file_tag = os.path.basename(os.path.dirname(input_file))
    file_tag = file_tag.replace('.SEN3','')
    output_file = input_file.replace(input_dir,output_dir)
    output_file = os.path.dirname(output_file).replace('.SEN3','_SUBSET_IDEPIX_C2RCC.nc')

    # the output file is named from the input file with the _C2RCCC suffix.
    # It will be a netCDF file in our example.
    print('Processing: '+str(os.path.basename(input_file)))
    print('To: '+str(os.path.basename(output_file)))
    
    # now adapt the config xml file for this specific input_file
    my_config = os.path.join(output_dir,'run_config.xml')
    if os.path.exists(my_config):
        os.remove(my_config)

    print('Generating config: '+str(my_config))
    shutil.copy(template_xml, my_config)

    # Read in the file
    with open(my_config, 'r') as file :
        filedata = file.read()
    # Replace the target string
    filedata = filedata.replace('SOURCE_PRODUCT', input_file)
    # Write the file out again
    with open(my_config, 'w') as file:
        file.write(filedata)

    # Read in the file
    with open(my_config, 'r') as file :
        filedata = file.read()
    # Replace the target string
    filedata = filedata.replace('OUTPUT_PRODUCT', output_file)
    # Write the file out again
    with open(my_config, 'w') as file:
        file.write(filedata)

    # the processing call is a follows below.
    c2rcc_processingCall = GPTPATH + ' ' + my_config

    # useful to check that the command call is correct before launching the call (comment / uncomment the next line)
    print('Config ready; running: ' + c2rcc_processingCall)

    # python call, uncomment when the printed call satisfies your requirements
    process = subprocess.Popen(c2rcc_processingCall, shell=True, stdout=subprocess.PIPE)
    process.wait()
    print('Finished running: ' + my_config)

Processing: xfdumanifest.xml
To: S3A_OL_1_EFR____20181027T082308_20181027T082608_20181028T140235_0179_037_192_3420_MAR_O_NT_002_SUBSET_IDEPIX_C2RCC.nc
Generating config: C:\Users\docle\Desktop\ocean-olly_wekeo_changes\GPT\GPT_L2_C2RCC_OUTPUT\run_config.xml
Config ready; running: "C:/Program Files\snap\bin\gpt.exe" C:\Users\docle\Desktop\ocean-olly_wekeo_changes\GPT\GPT_L2_C2RCC_OUTPUT\run_config.xml
Finished running: C:\Users\docle\Desktop\ocean-olly_wekeo_changes\GPT\GPT_L2_C2RCC_OUTPUT\run_config.xml
